NOMBRE: GOMEZ GARCIA FLORES DAFNE JULY
CI: 13119227
CU: 35-3800



Predicción de la calidad del agua      
Número de instancias (ejemplos): 705

Número de características (atributos): 11

Prediccion de la calidad de agua
contiene mediciones de calidad tomadas en 36 sitios de monitoreo en EE.UU
cada registro representa nuestra diaria del agua en un sitio especifico.
incluye 11 caracteristicas que representan diferentes aspectos de agua,como:
oxigeno disuelto, temperatura, conductividad y otros indicadores quimicos y fisicos.
este dataset cumple el predecir el valor de pH del agua para el dia siguiente, usando los calores actuales y pasados de las caracteristicas.
es un problema de regresion ya que el pH es un valor continuo.
este dataset se utiliza para modelos de prediccion de series de tiempo y aprendizaje automatico y regresion.
permite entrenar modelos para anticipar la calidad de agua y ayudar en gestion ambiental y decisiones de tratamiento de agua.


lo primero que hacemos es importas librerías: pandas, numpy y PyTorch para manejar datos, crear la red y entrenarla. También matplotlib para ver los resultados.

In [ ]:
# 1. Importar librerías
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
import matplotlib.pyplot as plt

Asi tambien empezamos a cargas el dataset water_quality.csv y revisas las primeras filas para familiarizarte con los datos.

In [7]:
# 2. Cargar dataset
df = pd.read_csv('/content/drive/MyDrive/PrediccionAgua.mat.csv')  # reemplaza con tu archivo
print("Primeras filas del dataset:")
print(df.head())

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/PrediccionAgua.mat.csv'

separamos las variables.

X: todas las características que describen la calidad del agua (excepto pH).

y: el valor de pH que queremos predecir.

In [ ]:
# 3. Separar características y variable objetivo
X = df.drop(columns=['pH']).values  # todas las columnas menos 'pH'
y = df['pH'].values.reshape(-1,1)   # variable a predecir

Se normaliza los datos entre 0 y 1 con MinMaxScaler, porque las redes aprenden mejor con valores pequeños y consistentes.

In [ ]:
# 4. Normalizar los datos
scaler_X = MinMaxScaler()
scaler_y = MinMaxScaler()
X_scaled = scaler_X.fit_transform(X)
y_scaled = scaler_y.fit_transform(y)


Creas secuencias de 7 días, usando los valores de los últimos 7 días para predecir el pH del día siguiente. Esto convierte tus datos en un problema de serie de tiempo.

In [ ]:
# 5. Crear secuencias de series de tiempo (7 días anteriores para predecir el siguiente)
def create_sequences(X, y, seq_length=7):
    X_seq, y_seq = [], []
    for i in range(len(X) - seq_length):
        X_seq.append(X[i:i+seq_length].flatten())  # aplanar características
        y_seq.append(y[i+seq_length])
    return np.array(X_seq), np.array(y_seq)

seq_length = 7
X_seq, y_seq = create_sequences(X_scaled, y_scaled, seq_length)

Conviertes los datos a tensores de PyTorch, que es el formato que necesita la red para entrenar.

In [ ]:
# 6. Convertir a tensores de PyTorch
X_tensor = torch.from_numpy(X_seq).float()
y_tensor = torch.from_numpy(y_seq).float()

Divides en entrenamiento y prueba, usando 80% para entrenar y 20% para evaluar, sin mezclar el orden porque es una serie temporal.

In [ ]:
# 7. Dividir en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X_tensor, y_tensor, test_size=0.2, shuffle=False)
train_dataset = TensorDataset(X_train, y_train)
test_dataset = TensorDataset(X_test, y_test)
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

Defines la red neuronal:

Capa de entrada (fc1) → 64 neuronas

Capa oculta (fc2) → 32 neuronas

Capa de salida → 1 neurona (predicción del pH)

Función ReLU para introducir no linealidad.

In [ ]:
# 8. Definir la red neuronal
class WaterQualityNN(nn.Module):
    def __init__(self, input_size):
        super(WaterQualityNN, self).__init__()
        self.fc1 = nn.Linear(input_size, 64)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 1)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.fc3(x)
        return x

input_size = X_seq.shape[1]  # número de características * sec_length
model = WaterQualityNN(input_size)

Configuras el entrenamiento:

MSELoss para medir error cuadrático medio.

Adam como optimizador para ajustar los pesos de la red.

In [ ]:
# 9. Configurar pérdida y optimizador
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

Entrenas la red durante 150 épocas. Para cada batch de datos, calculas predicción, pérdida, y ajustas los pesos.

In [ ]:
# 10. Entrenamiento de la red
epochs = 150
for epoch in range(epochs):
    model.train()
    for batch_X, batch_y in train_loader:
        optimizer.zero_grad()
        outputs = model(batch_X)
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()
    if (epoch+1) % 30 == 0:
        print(f'Epoch {epoch+1}/{epochs}, Loss: {loss.item():.6f}')

Evalúas la red en los datos de prueba y transformas los valores a la escala original de pH.

In [ ]:
# 11. Evaluación y predicción
model.eval()
predictions = model(X_test).detach().numpy()
predictions_rescaled = scaler_y.inverse_transform(predictions)
y_test_rescaled = scaler_y.inverse_transform(y_test.numpy())

Visualizas los resultados en un gráfico comparando pH real y pH predicho, para ver qué tan bien aprendió la red.

In [ ]:
# 12. Visualizar resultados
plt.figure(figsize=(12,6))
plt.plot(y_test_rescaled, label='pH real')
plt.plot(predictions_rescaled, label='pH predicho')
plt.xlabel('Día')
plt.ylabel('pH')
plt.title('Predicción de pH del agua')
plt.legend()
plt.show()